In [1]:
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [2]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING", 
    "WALKING_UPSTAIRS", 
    "WALKING_DOWNSTAIRS", 
    "SITTING", 
    "STANDING", 
    "LAYING"
] 


## Let's start by downloading the data: 

In [3]:
## Note: Linux bash commands start with a "!" inside those "ipython notebook" cells
#
DATA_PATH = "data/"
#
#!pwd && ls
#os.chdir(DATA_PATH)
#!pwd && ls
#
#!python download_dataset.py
#
#!pwd && ls
#os.chdir("..")
#!pwd && ls
#
DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)
#


Dataset is now located at: data/UCI HAR Dataset/


## Preparing dataset:

In [27]:
TRAIN = "train/"
TEST = "test/"

X_train_signals_paths = [DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES]
'''
['data/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt',
 'data/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt']
'''

X_test_signals_paths = [DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES]
'''
['data/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt',
 'data/UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt']
'''
##################################################
# Load "X" (the neural network's training and testing inputs)
##################################################
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append([np.array(serie, dtype=np.float32) for serie in [ row.replace('  ', ' ').strip().split(' ') for row in file]])
        file.close()
    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train = load_X(X_train_signals_paths)  # (7352, 128, 9)
X_test = load_X(X_test_signals_paths)    # (2947, 128, 9)



##################################################
# Load "y" (the neural network's training and testing outputs)
##################################################

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array([elem for elem in [row.replace('  ', ' ').strip().split(' ') for row in file]], dtype=np.int32)
    
    file.close()
    
    # Substract 1 to each output class for friendly 0-based indexing 
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
''' data/UCI HAR Dataset/train/y_train.txt '''

y_test_path = DATASET_PATH + TEST + "y_test.txt"
''' data/UCI HAR Dataset/test/y_test.txt '''


y_train = load_y(y_train_path)  # (7352, 1)
y_test = load_y(y_test_path)    # (2947, 1)

## Additionnal Parameters:

Here are some core parameter definitions for the training. 

For example, the whole neural network's structure could be summarised by enumerating those parameters and the fact that two LSTM are used one on top of another (stacked) output-to-input as hidden layers through time steps. 

In [5]:
# Input Data 

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) 0.09913992 0.39567086
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [6]:
print(('X_train: {}').format(X_train.shape))
print(('X_test: {}').format(X_test.shape))
print(('y_train: {}').format(y_train.shape))
print(('y_test: {}').format(y_test.shape))


##128: readings/window
##[acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z", "total_acc_x", "total_acc_y" , "total_acc_z"]
##
##["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"] 

X_train: (7352, 128, 9)
X_test: (2947, 128, 9)
y_train: (7352, 1)
y_test: (2947, 1)


In [7]:
X_train[0][0]

array([ 1.808515e-04,  1.076681e-02,  5.556068e-02,  3.019122e-02,
        6.601362e-02,  2.285864e-02,  1.012817e+00, -1.232167e-01,
        1.029341e-01], dtype=float32)

In [8]:
X_train[0][1]

array([ 0.01013856,  0.00657948,  0.05512483,  0.04371071,  0.04269897,
        0.01031572,  1.022833  , -0.1268756 ,  0.1056872 ], dtype=float32)

In [9]:
X_train[0][0]

array([ 1.808515e-04,  1.076681e-02,  5.556068e-02,  3.019122e-02,
        6.601362e-02,  2.285864e-02,  1.012817e+00, -1.232167e-01,
        1.029341e-01], dtype=float32)

## Utility functions for training:

In [10]:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters. 
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network. 
    # Note, some code of this notebook is inspired from an slightly different 
    # RNN architecture used on another dataset, some of the credits goes to 
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier, 
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s  # (1500, 128, 9)


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes 
    # e.g.: 
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


## Let's get serious and build the neural network:

In [11]:
################
# n_steps: 128   readings / window
# n_input: 9    [acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z", "total_acc_x", "total_acc_y" , "total_acc_z"]

# n_classes: 6  ["WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS", "SITTING", "STANDING", "LAYING"] 
# n_hidden: 32

#training_data_count: 7352
#test_data_count: 2947

#learning_rate: 0.0025
#lambda_loss_amount: 0.0015
# training_iters: 2205600

#batch_size: 1500
#display_iter: 30000

################


# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])  
y = tf.placeholder(tf.float32, [None, n_classes])         

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
weights

{'hidden': <tf.Variable 'Variable:0' shape=(9, 32) dtype=float32_ref>,
 'out': <tf.Variable 'Variable_1:0' shape=(32, 6) dtype=float32_ref>}

In [13]:
# prediction
pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation:
#################################

# L2 loss prevents this overkill neural network to overfit the data
l2 = lambda_loss_amount * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 

# Softmax loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 

# Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [25]:
step = 1
batch_xs =         extract_batch_size(X_train, step, batch_size)   
batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))  

# extract_batch_size(X_train, 1, 1500)             >>>>>  shape of output: (7352, 128, 9)
# extract_batch_size(y_train, 1, 1500)             >>>>>  shape of output: (1500, 1)
# one_hot(extract_batch_size(y_train, 1, 1500))    >>>>>  shape of output: (1500, 6)

print(X_train.shape)   # (7352, 128, 9)
print(y_train.shape)   # (7352, 1)
print(batch_size)      # 1500

print(batch_xs.shape)  # (1500, 128, 9)
print(batch_ys.shape)  # (1500, 6)
print(extract_batch_size(y_train, step, batch_size).shape)

(7352, 128, 9)
(7352, 1)
1500
(1500, 128, 9)
(1500, 6)
(1500, 1)
